In [1]:
import os

cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /Users/tarasbohdan/Desktop/Experiments-AAAI-24


In [2]:
%matplotlib inline

import sys
sys.path.append("../")
import copy

from aif360.datasets import BinaryLabelDataset
from aif360.datasets import AdultDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult
from aif360.algorithms.preprocessing.lfr import LFR

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

from IPython.display import Markdown, display
import matplotlib.pyplot as plt
import numpy as np


pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'


In [3]:
class Config:
  file_name = './data/student_performance/student-mat.csv'

In [4]:
from notebooks.mult_repair_levels_student_performance.StudentPerformanceDataset import StudentPerformanceDataset

data_loader = StudentPerformanceDataset(Config.file_name)
data_loader.X_data.head()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,Mjob,Fjob,reason,guardian,sex
0,18,4,4,2,2,0,4,3,4,1,1,3,6,0,0,at_home,teacher,course,mother,F
1,17,1,1,1,2,0,5,3,3,1,1,3,4,0,0,at_home,other,course,father,F
2,15,1,1,1,2,3,4,3,2,2,3,3,10,0,0,at_home,other,other,mother,F
3,15,4,2,1,3,0,3,2,2,1,1,5,2,1,1,health,services,home,mother,F
4,16,3,3,1,2,0,4,3,2,1,2,5,4,0,1,other,other,home,father,F


In [5]:
init_data_loader = copy.deepcopy(data_loader)

In [6]:
from virny.preprocessing import preprocess_dataset
from source.preprocessing import get_simple_preprocessor

data_loader.categorical_columns = [col for col in data_loader.categorical_columns if
                                   col not in ('sex')]
data_loader.X_data['sex_binary'] = data_loader.X_data['sex'].apply(lambda x: 1 if x == 'M' else 0)
data_loader.full_df = data_loader.full_df.drop(['sex'], axis=1)
data_loader.X_data = data_loader.X_data.drop(['sex'], axis=1)

# Preprocess the dataset using the defined preprocessor
column_transformer = get_simple_preprocessor(data_loader)
base_flow_dataset = preprocess_dataset(data_loader, column_transformer, 0.3, 42)
base_flow_dataset.init_features_df = init_data_loader.full_df.drop(init_data_loader.target, axis=1,
                                                                   errors='ignore')
base_flow_dataset.X_train_val['sex_binary'] = data_loader.X_data.loc[
    base_flow_dataset.X_train_val.index, 'sex_binary']
base_flow_dataset.X_test['sex_binary'] = data_loader.X_data.loc[base_flow_dataset.X_test.index, 'sex_binary']

/var/folders/cg/l243p36n10s5v1x_vrt2wzs80000gn/T/ipykernel_64506/2079174263.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_loader.X_data['sex_binary'] = data_loader.X_data['sex'].apply(lambda x: 1 if x == 'M' else 0)
/Users/tarasbohdan/Desktop/Experiments-AAAI-24/virny_venv/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [7]:
from sklearn.model_selection import train_test_split

full_df = data_loader.X_data

dataset_orig_train, dataset_orig_test = train_test_split(full_df, test_size=0.3)

In [8]:
sensitive_attribute = 'sex_binary'

In [9]:
train_df = base_flow_dataset.X_train_val
train_df[base_flow_dataset.target] = base_flow_dataset.y_train_val

test_df = base_flow_dataset.X_test
test_df[base_flow_dataset.target] = base_flow_dataset.y_test

In [10]:
train_binary_dataset = BinaryLabelDataset(df=train_df,
                                              label_names=[data_loader.target],
                                              protected_attribute_names=[sensitive_attribute],
                                              favorable_label=1,
                                              unfavorable_label=0)
test_binary_dataset = BinaryLabelDataset(df=test_df,
                                         label_names=[data_loader.target],
                                         protected_attribute_names=[sensitive_attribute],
                                         favorable_label=1,
                                         unfavorable_label=0)

In [11]:
# Metric for the original dataset
privileged_groups = [{'sex_binary': 1}]
unprivileged_groups = [{'sex_binary': 0}]

metric_orig_train = BinaryLabelDatasetMetric(train_binary_dataset,
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())
metric_orig_test = BinaryLabelDatasetMetric(test_binary_dataset,
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original test dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_test.mean_difference())

#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.034286


#### Original test dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.154042


In [12]:
TR = LFR(unprivileged_groups=unprivileged_groups,
         privileged_groups=privileged_groups,
         k=3, Ax=0.1, Ay=0.5, Az=4.0,
         verbose=1
        )
TR = TR.fit(train_binary_dataset, maxiter=5000, maxfun=5000)

step: 0, loss: 0.7430054200325741, L_x: 1.420629279221906,  L_y: 0.8264132570182177,  L_z: 0.04693396590031864
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          102     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  7.43005D-01    |proj g|=  2.66210D-01

At iterate    1    f=  6.25800D-01    |proj g|=  1.03889D-01
step: 250, loss: 0.5595164928447789, L_x: 1.427293837833227,  L_y: 0.654619986439549,  L_z: 0.02236927896042042

At iterate    2    f=  5.59516D-01    |proj g|=  1.28598D-01
step: 500, loss: 0.5073393855144686, L_x: 1.427457465245675,  L_y: 0.6654308003439587,  L_z: 0.007969559704480414

At iterate    3    f=  5.07339D-01    |proj g|=  1.92568D-01
step: 750, loss: 0.48929696679366136, L_x: 1.426883668337707,  L_y: 0.6651215090570112,  L_z: 0.003511961357846279

At iterate    4    f=  4.89297D-01    |proj g|=  1.99025D-01

At iterate    5    f=  4.62329D-01    |proj g|=  1.16639D-01
step: 100

In [13]:
dataset_transf_train = TR.transform(train_binary_dataset)
dataset_transf_test = TR.transform(test_binary_dataset)

In [14]:
metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train,
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())
metric_transf_test = BinaryLabelDatasetMetric(dataset_transf_test,
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Transformed test dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_test.mean_difference())

#### Transformed training dataset

Difference in mean outcomes between unprivileged and privileged groups = 0.016508


#### Transformed test dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.103166


In [15]:
train_df, _ = train_binary_dataset.convert_to_dataframe()
test_df, _ = test_binary_dataset.convert_to_dataframe()

In [16]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(random_state=42)

In [17]:
train_X = train_df.drop(['G3'], axis=1)
train_y = train_df['G3']

In [18]:
test_X = test_df.drop(['G3'], axis=1)
test_y = test_df['G3']

In [19]:
rfc = rfc.fit(train_X, train_y)

In [20]:
y_pred = rfc.predict(test_X)

In [21]:
report = classification_report(test_y, y_pred)
print("Classification Report:\n", report)

Classification Report:
               precision    recall  f1-score   support

         0.0       0.85      0.96      0.90        46
         1.0       0.97      0.89      0.93        73

    accuracy                           0.92       119
   macro avg       0.91      0.92      0.91       119
weighted avg       0.92      0.92      0.92       119



In [22]:
train_df, _ = dataset_transf_train.convert_to_dataframe()
test_df, _ = dataset_transf_test.convert_to_dataframe()


rfc = RandomForestClassifier(random_state=42)

train_X = train_df.drop(['sex_binary', 'G3'], axis=1)
train_y = train_df['sex_binary']

test_X = test_df.drop(['sex_binary', 'G3'], axis=1)
test_y = test_df['sex_binary']

rfc = rfc.fit(train_X, train_y)

y_pred = rfc.predict(test_X)

report = classification_report(test_y, y_pred)
print("Classification Report:\n", report)

Classification Report:
               precision    recall  f1-score   support

         0.0       0.51      0.62      0.56        58
         1.0       0.54      0.43      0.48        61

    accuracy                           0.52       119
   macro avg       0.52      0.52      0.52       119
weighted avg       0.52      0.52      0.52       119

